In [ ]:
import logging

import nemo.collections.asr as nemo_asr
import pandas as pd
import torch
from nemo.collections.nlp.models import PunctuationCapitalizationModel
from pyannote.audio import Pipeline
from pydub import AudioSegment
from pydub.silence import detect_silence

from ctcdecode import CTCBeamDecoder

logging.getLogger("nemo_logger").setLevel(logging.ERROR)
asr_logger = logging.getLogger("asr")
asr_logger.setLevel(logging.INFO)


In [ ]:
from asr import transcribe_mono

In [ ]:
transcription = transcribe_mono(
    "../output/radio_national_podcasts/audio/rewilding-the-scottish-highlands.mp3"
)
transcription

In [ ]:
transcription

In [ ]:
time_formatted_words_all = []
for idx, record in transcription.head(10).iterrows():
    time_formatted_words = _format_word_timestamps(record.asr_outputs, record.start)

    # 5.0 apply punctuation to each output
    punctuated_sequence = punct_model.add_punctuation_capitalization(
        [" ".join(e["word"] for e in time_formatted_words)]
    )[0]

    # if len(punctuated_sequence.split(" ")) == len(time_formatted_words):
    #     # easy case, where punctuated output len matches input len; assign directly
    #     punctuated_sequence_joined = (
    #         pd.DataFrame(time_formatted_words)
    #         .assign(word=punctuated_sequence.split(" "))
    #         .assign(speakerTag=record.speaker)
    #         .to_dict(orient="records")
    #     )
    #     time_formatted_words_all.append(punctuated_sequence_joined)
    # else:
    #     # otherwise.. pad the difference? changes should be limited to immediately proceeding fullstops, commas, question marks
    #     # https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/main/nlp/punctuation_and_capitalization.html
    #     print("Punctuated outputs not the same length as input")